In [19]:
import torch
import pandas as pd
import numpy as np
import sklearn

from sklearn import linear_model
from sklearn.svm import OneClassSVM
from sklearn.linear_model import SGDOneClassSVM
from sklearn.metrics import f1_score, confusion_matrix, classification_report
from sklearn.kernel_approximation import Nystroem
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import LocalOutlierFactor

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings(action='ignore')

In [20]:
train = pd.read_csv('../dataset/train.csv')
val = pd.read_csv('../dataset/val.csv')
test = pd.read_csv('../dataset/test.csv')

In [21]:
val_normal, val_abnormal = val.Class.value_counts()
val_ratio = val_abnormal / val_normal
print(val_normal, val_abnormal, val_ratio)


28432 30 0.0010551491277433877


In [22]:
train_x = train.drop(columns=['ID'])
val_x = val.drop(columns=['ID', 'Class'])
val_y = val['Class']

In [17]:
# clf = LocalOutlierFactor(n_neighbors=2, novelty=True)
# clf.fit(train_x)
# pred_y = clf.predict(val_x)

In [64]:
# fit the model
# clf = OneClassSVM(nu=val_ratio, kernel="rbf", gamma=0.1)
clf = OneClassSVM(nu=val_ratio, kernel="poly", gamma='scale')
clf.fit(train_x)


OneClassSVM(kernel='poly', nu=0.0010551491277433877)

In [65]:
def get_pred(model, x, k) :
    prob = model.score_samples(x)
    prob = torch.tensor(prob, dtype=torch.float)
    topk_indices = torch.topk(prob, k=k, largest=False).indices
    
    pred = torch.zeros(len(x), dtype=torch.int8)
    pred[topk_indices] = 1
    
    return pred.tolist(), prob.tolist()

In [89]:
val_pred, val_prob = get_pred(clf, val_x, 29)

In [90]:
val_score = f1_score(val_y, val_pred, average='macro')
val_score

0.5164479655204789

In [91]:
print(classification_report(val_y, val_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28432
           1       0.03      0.03      0.03        30

    accuracy                           1.00     28462
   macro avg       0.52      0.52      0.52     28462
weighted avg       1.00      1.00      1.00     28462



In [92]:
confusion_matrix(val_y, val_pred)

array([[28404,    28],
       [   29,     1]])

In [93]:
# val_pred

In [96]:
np.where(np.array(val_pred) == 1)

(array([  477,   764,   918,  3666,  5602,  5864,  5882,  7541,  9890,
        12797, 13539, 14767, 14833, 14889, 15092, 17082, 17767, 18238,
        20460, 20836, 21607, 22646, 22856, 23407, 24100, 24907, 26018,
        27913, 28414]),)